In [1]:
import numpy as np
import time
import pandas as pd
from bs4 import BeautifulSoup
import cloudscraper
from concurrent.futures import ThreadPoolExecutor
import random

In [8]:
def get_links(page):
    time.sleep(1)
    page= str(page+1)
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }

    link ="https://www.nhatot.com/mua-ban-can-ho-chung-cu-ha-noi?page="+page

    # Gửi yêu cầu HTTP bằng requests
    #r = requests.get(link, headers=headers)
    try: 
        response = scraper.get(link, headers=headers)
        if response.status_code == 200:
        # Phân tích nội dung HTML bằng BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tìm tất cả các thẻ <a> có class 'crd7gu7'
            a = soup.find_all("a", itemprop="item")

        # Lấy href từ các thẻ <a> và thêm vào danh sách
            for i in range(len(a)):
                link = "https://www.nhatot.com/" + a[i]['href']
                arr.append(link)
        else:
            print(f"Failed to fetch page {page}, status code: {response.status_code}")
            return []
    except Exception as error:
        print(error)
        return []


arr = []
scraper = cloudscraper.create_scraper()

start_time = time.time()
with ThreadPoolExecutor(max_workers=10) as executor:
    # Chạy đa luồng với 296 trang
    results = list(executor.map(get_links, range(296)))
    
for result in results:
    if result is not None:
        arr.extend(result)    
# Tạo DataFrame từ danh sách các link
df_links = pd.DataFrame(arr,columns=['links'])

Failed to fetch page 2, status code: 403
Failed to fetch page 136, status code: 403
Failed to fetch page 138, status code: 403
Failed to fetch page 140, status code: 403
Failed to fetch page 144, status code: 403
Failed to fetch page 148, status code: 403
Failed to fetch page 163, status code: 403
Failed to fetch page 183, status code: 403
Failed to fetch page 215, status code: 403
Failed to fetch page 234, status code: 403


In [ ]:
df_links.to_csv('chotot_2.csv',index=True)
print("done")

In [2]:
frame = pd.read_csv('chotot_2.csv')
frame.head()

,Unnamed: 0,links
0,0,https://www.nhatot.com/https://www.nhatot.com
1,1,https://www.nhatot.com/https://www.nhatot.com/...
2,2,https://www.nhatot.com//mua-ban-can-ho-chung-c...
3,3,https://www.nhatot.com//mua-ban-can-ho-chung-c...
4,4,https://www.nhatot.com//mua-ban-can-ho-chung-c...


In [3]:
address = []
estate_condition = []
acreage = []
price_m2 = []
bedroom = []
block = []
floor = []
bath_room = []
types = []
Legal_papers = []
id_house = []
furniture_condition = []
main_direction = []
balcony_direction = []
Character =  []
price = []

In [5]:
# Từ điển ánh xạ giữa các thuộc tính và danh sách tương ứng
mapping_dict = {
    'Tình trạng bất động sản': estate_condition,
    'Diện tích': acreage,
    'Giá/m²': price_m2,  # Sử dụng đúng ký tự 'm²'
    'Số phòng ngủ': bedroom,
    'Tên phân khu/Lô/Block/Tháp': block,
    'Tầng số': floor,
    'Số phòng vệ sinh': bath_room,
    'Loại hình căn hộ': types,
    'Giấy tờ pháp lý': Legal_papers,
    'Mã căn': id_house,
    'Tình trạng nội thất': furniture_condition,
    'Hướng cửa chính': main_direction,
    'Hướng ban công': balcony_direction,
    'Đặc điểm căn hộ': Character
}

def get_data(link):
    try:
        time.sleep(random.uniform(2, 5))  
        info_class = "col-xs-6 AdParam_adParamItemPty__k3rlZ"
        notfound_class = "notfound-message"

        # Gửi yêu cầu HTTP
        headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0'
        }

        
        scraper = cloudscraper.create_scraper()
        r = scraper.get(link, headers=headers)
        print(r)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        if soup.find(class_=notfound_class) is None:
            # Lấy địa chỉ (nếu có)
            add = soup.find(class_='bwq0cbs flex-1')
            address.append(add.text.strip() if add else np.nan)

            # Lấy giá (nếu có)
            
            price_tag = soup.find(class_='pyhk1dv')
            price.append(price_tag.text.strip() if price_tag else np.nan)

            flags = {key: False for key in mapping_dict.keys()}

            # Tìm tất cả các thẻ chứa thông tin
            items = soup.find_all(class_=info_class)
            # Duyệt qua các phần tử thông tin
            for item in items:
                label = item.find('div', class_='AdParam_adParamLabelPty__Qp6G3').get_text(strip=True)
                value = item.find('strong', class_='AdParam_adParamValuePty__3uTmt').get_text(strip=True)
                for key, lst in mapping_dict.items():
                    if label == key:
                        lst.append(value)
                        flags[key] = True

            # Điền NaN cho các trường không tìm thấy
            for key, lst in mapping_dict.items():
               if not flags[key]:  # Nếu không tìm thấy thông tin cho trường, thêm NaN
                    lst.append(np.nan)

    except Exception as e:
        print(f"Error processing link {link}: {e}")

In [6]:
get_data('https://www.nhatot.com//mua-ban-can-ho-chung-cu-quan-hoang-mai-ha-noi/121909319.htm')

<Response [429]>


In [6]:
with ThreadPoolExecutor(max_workers=10) as executor:  # Tối đa 5 luồng
    executor.map(get_data, frame['links'])

<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [403]>
<Response [403]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>


KeyboardInterrupt: 

<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [429]>
<Response [403]>
<Response [429]>
<Response [429]>
<Response [429]>


In [6]:
dict_data = {
    'DiaChi' : address,
    'TinhTrangBDS' : estate_condition,
    'DienTich' : acreage,
    'Gia/m2'   : price_m2,
    'Phongngu': bedroom,
    'TenPhanKhu': block,
    'SoTang': floor,
    'PhongTam':bath_room,
    'Loai': types,
    'GiayTo': Legal_papers,
    'MaCanHo': id_house,
    'TinhTrangNoiThat': furniture_condition,
    'HuongCuaChinh': main_direction,
    'HuongBanCong': balcony_direction,
    'DacDiem':Character,
    'Gia': price,
}
df_data = pd.DataFrame(dict_data)
df_data.to_csv('data_crawl.csv',encoding="utf-8-sig",index=False)